# 1. Neural Networks
## 1.1 Visualizing the data

In [ ]:
import os
import random
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import scipy.optimize as opt
from pathlib import Path

data_file_1 = Path(os.path.abspath("")) / "data" / "ex4data1.mat"
data_set_1 = sio.loadmat(data_file_1)

# every row of X is a flattened 20 * 20 image matrix
# every row of y is a digit index, where [1, ..., 9] means digit [1, ..., 9], [10] means digit [0]
X = np.array(data_set_1["X"])
y = np.array(data_set_1["y"])

print(X)
print(y)

# transform y into Y
Y = np.zeros((y.shape[0], 10))
for i, v in enumerate(y):
    Y[i, v - 1] = 1

print(Y)

## 1.2 Model representation

In [ ]:
weights_file = Path(os.path.abspath("")) / "data" / "ex4weights.mat"
weights = sio.loadmat(weights_file)
theta_1 = weights["Theta1"]
theta_2 = weights["Theta2"]
print(theta_1.shape)
print(theta_2.shape)

## 1.3 Feedforward and cost function

In [ ]:
lamda = 0

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def h(theta_1, theta_2, x):
    a_2 = sigmoid(theta_1 @ x)
    a_3 = sigmoid(theta_2 @ np.insert(a_2, 0, 1, axis=0))
    return a_3

def regularization(theta_1, theta_2, lamda, m):
    regu_params = np.concatenate((theta_1[:, 1:].flatten(), theta_2[:, 1:].flatten()))
    sum = np.sum([i ** 2 for i in regu_params])
    res = lamda / (2 * m) * sum
    return res

def nnCostFunction(theta_1, theta_2, lamda, X, Y):
    input_X = np.insert(X, 0, np.ones(X.shape[0]), axis=1)
    m = Y.shape[0]
    cost = 0
    cost_regu = regularization(theta_1, theta_2, lamda, m)
    for i in range(m):
        x = input_X[i].reshape((input_X.shape[1], 1))
        y = Y[i].reshape((Y.shape[1], 1))
        H = h(theta_1, theta_2, x)
        cost_matrix = 1 / m * (- y * np.log(H) - (1 - y) * np.log(1 - H))
        cost += np.sum(cost_matrix)
    return cost + cost_regu

# cost when lamda = 0
print(nnCostFunction(theta_1, theta_2, lamda, X, Y))

## 1.4 Regularized cost function

In [ ]:
# cost when lamda = 1
print(nnCostFunction(theta_1, theta_2, 1, X, Y))